## Reaction Wheel Pendulum

### Equation of Motion

In [ ]:
from sympy import *
from sympy.physics.mechanics import *
init_vprinting()

In [ ]:
alpha, theta = dynamicsymbols("alpha, theta")
alphad, thetad = dynamicsymbols("alpha, theta", 1)
u1, u2 = dynamicsymbols("u1, u2")
Tm = dynamicsymbols("T_m")
Lg, L, g, m_pend, m_disk = symbols("L_G, L, g, m_p, m_d")

N, A, B = symbols("N, A, B", cls=ReferenceFrame)
A.orient_axis(N, N.z, theta)
B.orient_axis(A, A.z, alpha)

# O: pin joint of the pendulum
# G: center of mass of the rod
# P: pin joint of the wheel
O, P, G = symbols("O, P, G", cls=Point)
G.set_pos(O, -Lg * A.y)
P.set_pos(O, -L * A.y)

O.set_vel(N, 0)
O.set_vel(A, 0)
G.v2pt_theory(O, N, A)
G.a2pt_theory(O, N, A)
P.v2pt_theory(O, N, A)
P.a2pt_theory(O, N, A)

In [ ]:
Ixxp, Iyyp, Izzp = symbols("I_xx__p I_yy__p I_zz__p")
Ixxd, Iyyd, Izzd = symbols("I_xx__d I_yy__d I_zz__d")
# centrail inertias of the rod and disk
inertia_pend = inertia(A, Ixxp, Iyyp, Izzp)
inertia_disk = inertia(B, Ixxd, Iyyd, Izzd)

bodies = [
    RigidBody("pend", G, A, m_pend, (inertia_pend, G)),
    RigidBody("disk", P, B, m_disk, (inertia_disk, P))
]

In [ ]:
loads = [
    (G, -m_pend * g * N.y),
    (P, -m_disk * g * N.y),
    (B, Tm * N.z),
    (A, -Tm * N.z)
]

In [ ]:
kd_eqs = [
    thetad - u1,
    alphad - u2,
]
kane = KanesMethod(
    N,
    q_ind=[theta, alpha],
    u_ind=[u1, u2],
    kd_eqs=kd_eqs
)

fr, frstar = kane.kanes_equations(bodies, loads)
fr + frstar

### Optimization with Opty - Fixed duration simulation

In [ ]:
import opty
import numpy as np

#### Prepare the EoM 

In [ ]:
eom = Matrix.col_join(Matrix(kd_eqs), fr + frstar)
eom

#### Identify the known parameters, the control inputs and the state variables

In [ ]:
Lg_val, L_val, m_pend_val, m_disk_val, r_val = 1.25, 2, 1, 1, 0.5
known_parameter_map = {
    g: 9.81,
    Lg: Lg_val,
    L: L_val,
    m_pend: m_pend_val,
    m_disk: m_disk_val,
    Izzp: m_pend_val * L_val**2 / 12,
    Izzd: m_disk_val * r_val**2 / 2
}
known_parameter_map

In [ ]:
unknown_parameter = []
unknown_input_trajectories = [
    Tm
]
state_symbols = [
    theta, alpha, u1, u2
]

#### Decide between a fixed duration solution or variable duration solution

In [ ]:
num_nodes = 500
t0, tf = 0, 10
times = np.linspace(t0, tf, num_nodes)
h = (tf - t0) / (num_nodes - 1)

#### Create the objective function and its gradient

In [ ]:
opty.create_objective_function?

In [ ]:
t = symbols("t")
objective = Integral(Tm**2, t)
objective_func, objective_jacobian_func = opty.create_objective_function(
    objective,
    state_symbols,
    unknown_input_trajectories,
    unknown_parameter,
    num_nodes,
    h
)

#### Create the necessary constraints on the states

In [ ]:
theta_0 = 0
alpha_0 = 0
instance_constraints = [
    theta.subs(t, 0) - theta_0,
    theta.subs(t, tf) - pi,
    alpha.subs(t, 0) - alpha_0,
    u1.subs(t, 0),
    u1.subs(t, tf),
    u2.subs(t, 0),
    u2.subs(t, tf)
]

#### Create the necessary bounds to unknown parameters, unknown trajectories and states

In [ ]:
bounds = {
    Tm: (-10, 10),
}

#### Setup the optimization problem

In [ ]:
opty.Problem?

In [ ]:
problem = opty.Problem(
    objective_func,
    objective_jacobian_func,
    eom,
    state_symbols,
    num_nodes,
    h,
    known_parameter_map=known_parameter_map,
    instance_constraints=instance_constraints,
    bounds=bounds,
    time_symbol=t,
    integration_method="midpoint"
)
# optional step
problem.add_option('max_iter', 3000)

#### Create initial conditions

In [ ]:
initial_guess = np.random.randn(problem.num_free)
problem.plot_trajectories(initial_guess)

In [ ]:
problem.num_free

#### Run the optimization

In [ ]:
solution, info = problem.solve(initial_guess)

In [ ]:
print(info['status_msg'])
print(info['obj_val'])

#### Analyze the results

In [ ]:
problem.plot_trajectories(solution)
problem.plot_objective_value()

### Animation

In [ ]:
opty.utils.parse_free?

In [ ]:
state_traj, input_traj, constants = opty.utils.parse_free(
    solution, problem.collocator.num_states,
    problem.collocator.num_unknown_input_trajectories,
    problem.collocator.num_collocation_nodes,
    variable_duration=problem.collocator._variable_duration)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

xP = lambdify([L, theta], P.pos_from(O) & N.x)
yP = lambdify([L, theta], P.pos_from(O) & N.y)
rot = lambdify([alpha, theta], N.dcm(B))
points = np.array([[r_val, 0, 0], [0, 0, 0], [0, r_val, 0]])
points_on_wheel = np.array([[0, r_val, 0]])

def create_animation(state_traj):
    fig, ax = plt.subplots()
    
    idx0 = 0
    th0 = state_traj[0, idx0]
    al0 = state_traj[1, idx0]
    xP0 = xP(L_val, th0)
    yP0 = yP(L_val, th0)
    phi = np.linspace(0, 2*np.pi, 50, endpoint=True)
    x_wheel = r_val * np.cos(phi)
    y_wheel = r_val * np.sin(phi)
    points_on_wheel_transformed = (rot(al0, th0) @ points_on_wheel.T).T
    
    pendulum, = ax.plot([0, xP0], [0, yP0])
    wheel, = ax.plot(xP0 + x_wheel, yP0 + y_wheel)
    points_on_wheel_handler = ax.scatter(xP0 + points_on_wheel_transformed[:, 0], yP0 + points_on_wheel_transformed[:, 1])
    
    ax.set_aspect("equal")
    w = (L_val + r_val) * 1.25
    ax.axis([-w, w, -w, w])
    
    def update(idx):
        theta = state_traj[0, idx]
        alpha = state_traj[1, idx]
        xP_cur = xP(L_val, theta)
        yP_cur = yP(L_val, theta)
        points_on_wheel_transformed = (rot(alpha, theta) @ points_on_wheel.T).T
        
        pendulum.set_data([0, xP_cur], [0, yP_cur])
        wheel.set_data(xP_cur + x_wheel, yP_cur + y_wheel)
        points_on_wheel_handler.set_offsets(np.c_[xP_cur + points_on_wheel_transformed[:, 0], yP_cur + points_on_wheel_transformed[:, 1]])
        ax.set_title("t = {:.2f} s".format(times[idx]))
    
    # update(0)
    plt.close(fig)
    return FuncAnimation(fig, func=update, frames=len(times))

ani = create_animation(state_traj)
HTML(ani.to_jshtml(fps=30))

### Variable duration simulation

In [ ]:
num_nodes = 500
h = symbols("t")

In [ ]:
def objective_func(free):
    """Minimize the sum of the squares of the control torque."""
    T, h = free[4*num_nodes:-1], free[-1]
    return h * np.sum(T**2)

def objective_jacobian_func(free):
    T, h = free[4*num_nodes:-1], free[-1]
    grad = np.zeros_like(free)
    grad[4*num_nodes:-1] = 2 * h * T
    grad[-1] = np.sum(T**2)
    return grad

In [ ]:
tf = (num_nodes - 1) * h
theta_0 = 0
alpha_0 = 0
instance_constraints = [
    theta.subs(t, 0) - theta_0,
    theta.subs(t, tf) - pi,
    alpha.subs(t, 0) - alpha_0,
    u1.subs(t, 0),
    u1.subs(t, tf),
    u2.subs(t, 0),
    u2.subs(t, tf)
]

bounds = {
    Tm: (-10, 10),
    h: (0, 0.05)
}

In [ ]:
problem = opty.Problem(
    objective_func,
    objective_jacobian_func,
    eom,
    state_symbols,
    num_nodes,
    h,
    known_parameter_map=known_parameter_map,
    instance_constraints=instance_constraints,
    bounds=bounds,
    time_symbol=t
)
# optional step
problem.add_option('max_iter', 3000)

In [ ]:
max_attempts = 10
i = 0
converged = False
logs = {}

while (i < max_attempts) and (not converged):
    initial_guess = np.random.randn(problem.num_free)
    initial_guess[-1] = 1 / num_nodes
    solution, info = problem.solve(initial_guess)
    converged = not info["status"]
    logs[i] = info.copy()
    i += 1

In [ ]:
print(info["status_msg"])
print(info["obj_val"])

In [ ]:
problem.plot_trajectories(solution)
problem.plot_objective_value()

In [ ]:
state_traj, input_traj, constants, h_val = opty.utils.parse_free(
    solution, problem.collocator.num_states,
    problem.collocator.num_unknown_input_trajectories,
    problem.collocator.num_collocation_nodes,
    variable_duration=problem.collocator._variable_duration)

In [ ]:
ani = create_animation(state_traj)
HTML(ani.to_jshtml(fps=30))